In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

key = os.getenv("OPENROUTER_API_KEY")
print("KEY:", key)
print("Starts with sk-or-:", key.startswith("sk-or-") if key else None)


KEY: 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
Starts with sk-or-: True


In [3]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

# First API call with reasoning
response = client.chat.completions.create(
  model="xiaomi/mimo-v2-flash:free",
  messages=[
          {
            "role": "user",
            "content": "pick a business area that might be worth exploring for an Agentic AI combined with RAG system opportunity."
          }
        ],
  extra_body={"reasoning": {"enabled": True}}
)

# Extract the assistant message with reasoning_details
response = response.choices[0].message
print("First response:", response.content)

########################################################################################################
# Preserve the assistant message with reasoning_details
messages = [
  {"role": "user", "content": "pick a business area that might be worth exploring for an Agentic AI combined with RAG system opportunity."},
  {
    "role": "assistant",
    "content": response.content,
    "reasoning_details": response.reasoning_details  # Pass back unmodified
  },
  {"role": "user", "content": "Make sure it generates good amount of revenue."}
]

# Second API call - model continues reasoning from where it left off
response2 = client.chat.completions.create(
  model="xiaomi/mimo-v2-flash:free",
  messages=messages,
  extra_body={"reasoning": {"enabled": True}}
)

#########################################################################################################

messages = [
  {"role": "user", "content": "pick a business area that might be worth exploring for an Agentic AI combined with RAG system opportunity."},
  {
    "role": "assistant",
    "content": response.content,
    "reasoning_details": response.reasoning_details  # Pass back unmodified
  },
  {"role": "user", "content": "Make sure it generates good amount of revenue."},
  {
    "role": "assistant",
    "content": response2.choices[0].message.content,
    "reasoning_details": response2.choices[0].message.reasoning_details  # Pass back unmodified
  },
  {"role": "user", "content": "Now propose the agentic AI solution."}
]
# Third API call - model continues reasoning from where it left off
response3 = client.chat.completions.create(
  model="xiaomi/mimo-v2-flash:free",
  messages=messages,
  extra_body={"reasoning": {"enabled": True}}
)

First response: Here's a business area ripe for an Agentic AI + RAG system opportunity, with high potential impact and clear justification:

**Industrial Equipment Maintenance & Field Service Optimization**

**Why it's a perfect fit:**

1.  **High Complexity & Variability:** Industrial equipment (manufacturing lines, energy turbines, medical devices) involves intricate mechanics, electronics, and software. Failures are diverse and context-dependent.
2.  **Critical Knowledge Fragmentation:** Vital information lives in siloed sources: OEM manuals (dense PDFs), maintenance logs (unstructured text), sensor data streams, tribal knowledge from veteran technicians (often undocumented), parts databases, safety bulletins.
3.  **Urgent Need for Accurate Action:** Downtime is incredibly costly ($10k-$250k+ per hour). Technicians need precise, actionable instructions *fast*, often in stressful, noisy environments. Mistakes are expensive and dangerous.
4.  **Skilled Labor Shortage:** Aging workforc